### Домашняя работа №3

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [1]:
import pandas as pd

In [2]:
news = pd.read_csv('lenta-ru-news.csv.gz')
news

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество
...,...,...,...,...,...
739346,https://lenta.ru/news/1999/08/31/cholera/,Южно-Сахалинск объявлен очагом холеры,Сегодня областной центр Сахалина и Курил получ...,Россия,Все
739347,https://lenta.ru/news/1999/08/31/polit_blok/,Леворадикалы создают предвыборный блок,Бывший шеф Службы безопасности президента Але...,Россия,Все
739348,https://lenta.ru/news/1999/08/31/dagestan/,В горах Дагестана идут активные боевые действия,Сегодня утром в районах дагестанских селений Ч...,Россия,Все
739349,https://lenta.ru/news/1999/08/31/karachaevo/,Карачаево-Черкесия раскололась по национальном...,Намеченная на сегодняшний день церемония вступ...,Россия,Все


In [3]:
!pip install annoy

                                              0.0/647.5 kB ? eta -:--:--
     --                                    41.0/647.5 kB 653.6 kB/s eta 0:00:01
     ----------                             174.1/647.5 kB 2.1 MB/s eta 0:00:01
     ---------------------                  358.4/647.5 kB 2.8 MB/s eta 0:00:01
     -------------------------------------  645.1/647.5 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 647.5/647.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for annoy: filename=annoy-1.17.3-cp39-cp39-win_amd64.whl size=52783 sha256=45d6519b3d447ecfae09d3e3c83c156081796fafe173e5cac15cf386625cbfec
  Stored in directory: c:\users\leva_\appdata\local\pip\cache\wheels\09\a9\54\37478e65995fe712f7da465749da9ddb21db6b1a599d591ac7
Successfully built annoy


In [5]:
!pip install gensim

                                              0.0/24.0 MB ? eta -:--:--
                                              0.1/24.0 MB 2.6 MB/s eta 0:00:10
                                              0.3/24.0 MB 3.8 MB/s eta 0:00:07
     -                                        0.7/24.0 MB 5.3 MB/s eta 0:00:05
     --                                       1.2/24.0 MB 7.1 MB/s eta 0:00:04
     --                                       1.7/24.0 MB 7.9 MB/s eta 0:00:03
     --                                       1.8/24.0 MB 7.5 MB/s eta 0:00:03
     ---                                      2.1/24.0 MB 6.5 MB/s eta 0:00:04
     -----                                    3.3/24.0 MB 9.3 MB/s eta 0:00:03
     -----                                    3.6/24.0 MB 9.1 MB/s eta 0:00:03
     ------                                   3.7/24.0 MB 7.8 MB/s eta 0:00:03
     --------                                 5.0/24.0 MB 9.9 MB/s eta 0:00:02
     ---------                                5.5/24.0 MB 1

In [6]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import tqdm

In [7]:
list_news = news.text.to_list()
list_news[:2]

['Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 'Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо 

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in list_news:
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 1000:
        break

In [10]:
sentences = [i for i in sentences if len(i) > 2]

In [11]:
sentences[0][:20]

['вицепремьер',
 'социальный',
 'вопрос',
 'татьяна',
 'голиков',
 'рассказать',
 'регион',
 'россия',
 'зафиксировать',
 'высокий',
 'смертность',
 'рак',
 'сообщать',
 'риа',
 'новость',
 'слово',
 'голиков',
 'частый',
 'онкологический',
 'заболевание']

In [15]:
Word2Vec?

In [19]:
%%time
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)

CPU times: total: 1.69 s
Wall time: 726 ms


In [20]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for question in sentences:
    n_w2v = 0
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    w2v_index.add_item(counter, vector_w2v)
        
    counter += 1
    
    if counter > 1000:
        break

w2v_index.build(10)

True

In [21]:
def get_response(question, index, model, index_map, num_question):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, num_question, )
    return [list_news[i] for i in answers]

In [30]:
TEXT = "Интернет"
get_response(TEXT, w2v_index, modelW2V, index_map, 3)

['В сети появилось шуточное видео, на котором Кит Харрингтон, исполняющий роль Джона Сноу в сериале «Игра престолов», якобы проходит кастинг для участия в новом проекте. Ролик опубликован на YouTube-канале Universal Pictures Russia. Вместе с Харрингтоном на съемочной площадке появляется Беззубик — персонаж анимационного фильма «Как приручить дракона». На видео звезда «Игры престолов» зачитывает сценарий и пытается вжиться в свою роль. Он подкармливает сказочное чудовище лососем и пытается обучить его актерскому мастерству. В феврале 2019 года в России выйдет мультфильм «Как приручить дракона 3». В нем Кит Харрингтон озвучивает героя по имени Эрин. 5 декабря актер рассказал, что окончание съемок финальной части сериала «Игра престолов» стало для него сильным эмоциональным ударом. При этом артист заявил, что ни за что бы не стал возвращаться в проект.',
 'Телеканал «Дождь» выпустил репортаж о деятельности кибердружинников, которые, сотрудничая с правоохранительными органами, занимаются в

In [35]:
TEXT = "Россия"
get_response(TEXT, w2v_index, modelW2V, index_map, 3)

['Президент России Владимир Путин спародировал своего предшественника Бориса Ельцина, рассказывая, как тот предложил ему стать преемником. Видео из выступления Путина на форуме «Проектория» в Ярославле опубликовано в Telegram-канале «Проект Сноб». Путин рассказал, что Ельцин вызвал его к себе и сообщил, что хочет представить в Госдуме его кандидатуру на пост премьер-министра, а потом собирается предложить ему идти на выборы президента. Ельцин спросил, согласен ли Путин, и тот ответил, что нет. «Он очень удивился, он был такой человек, своеобразный, [и говорит]: "Почему это"?» — сказал Путин, сымитировав интонацию Ельцина. По словам нынешнего главы государства, тогда он ответил, что не готов, хотя «уже был большой мальчик». В конце выступления Путин пожелал всем собравшимся быть готовыми как можно раньше принять ключевое решение в своей жизни.',
 'Президент Украины Петр Порошенко на совместной с вице-президентом Европейской комиссии Валдисом Домбровскисом пресс-конференции в Брюсселе пе

In [36]:
TEXT = "искусственный интелект"
get_response(TEXT, w2v_index, modelW2V, index_map, 3)

['Искусственный интеллект компьютерной игры Eve Online вышел из-под контроля и устроил битву трех космических флотилий без участия человека. Об этом пишет портал 3Dnews. Выяснилось, что фракция шахтеров подверглась атаке отряда пиратов, вызвала на защиту флот «Амарр», а позднее к сражению подключилась фракция «Скитальцы». В результате союза и последовавшего предательства пираты вышли победителями. Отличает эту битву то, что она проходила без участия человека, силами только искусственного интеллекта. Живых свидетелей сражения нет. Пользователи узнали о случившемся из логов уничтоженных кораблей. Компания CCP Games расширила возможности неигровых персонажей EVE Online еще в 2017 году. Теперь разработчики расследуют ситуацию и намерены больше узнать о действиях и рисках неуправляемого искусственного интеллекта. EVE Online, космическая MMORPG, в которой различные кланы и фракции сражаются за доминирование в галактике, существует с 2003 года. В ней присутствуют пять основных фракций. Игрок 

In [33]:
TEXT = "нейронная сеть"
get_response(TEXT, w2v_index, modelW2V, index_map, 3)

['Фотография победительницы конкурса «Миссис Москва» заставила пользователей социальных сетей усомниться в красоте участницы. Ее снимок разместил на своем Facebook блогер Рустем Адагамов. Комментаторы заподозрили победительницу в злоупотреблении ботоксом и предположили, что на выбор жюри мог повлиять денежный взнос. «Кто же за нее так хорошо заплатил жюри, что они ее якобы выбрали из всех претенденток? Тариф бы указали, чтобы будущие претендентки уже знали, смогут ли они финансово в этом поучаствовать!» — написала одна из пользовательниц. В Instagram победительницы также появились комментарии от ненавистников, один из них окрестил женщину «чудовищем ботоксным». В ответ на это она опубликовала пост, в котором призвала людей «жить и радоваться», а не «наблюдать за чужим счастьем». Сам конкурс, как сообщает «Экспресс-газета», прошел 2 декабря. Победительницей среди «самых активных жен и мам столицы» стала 35-летняя Екатерина Лифшиц, мать двух мальчиков-близнецов. До рождения детей женщина

Не все запросы отработали корректно, получилось что чем "проще запрос", тем лучше выдачи, либо недостаточно статей...